In [1]:
!python -V

Python 3.9.19


In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb

from sklearn.metrics import mean_squared_error, root_mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle

In [3]:
import mlflow

mlflow.set_tracking_uri("./mlruns")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/mlruns/473165467644918906', creation_time=1716218234850, experiment_id='473165467644918906', last_update_time=1716218234850, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
from mlflow.tracking import MlflowClient

# Initialize the MLflow client
client = MlflowClient()

# Get a list of all experiments
experiments = client.search_experiments()

# Print the details of each experiment
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print(f"Lifecycle Stage: {experiment.lifecycle_stage}")
    print()

Experiment ID: 473165467644918906
Name: nyc-taxi-experiment
Artifact Location: /Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/mlruns/473165467644918906
Lifecycle Stage: active



In [10]:
Experiment_ID_to_delete = "407136421235301508"

# Delete the experiment
client.delete_experiment(Experiment_ID_to_delete)


In [11]:
# Get a list of all experiments
experiments = client.search_experiments()

# Print the details of each experiment
for experiment in experiments:
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print(f"Lifecycle Stage: {experiment.lifecycle_stage}")
    print()

Experiment ID: 473165467644918906
Name: nyc-taxi-experiment
Artifact Location: /Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/mlruns/473165467644918906
Lifecycle Stage: active



In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715208946364

In [12]:
# save the model - as linear regression gives the best results
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [13]:
lr = Lasso()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

12.212583224318818

In [14]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

11.167275941179728

In [15]:
with mlflow.start_run():

    mlflow.set_tag("develop", "R.Sanghvi")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.0001
    lr = Lasso(alpha)
    mlflow.log_param("alpha", alpha)

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)


In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [16:49:35] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.59415                          
[1]	validation-rmse:11.03834                          
[2]	validation-rmse:10.53690                          
[3]	validation-rmse:10.08966                          
[4]	validation-rmse:9.68559                           
[5]	validation-rmse:9.32998                           
[6]	validation-rmse:9.00324                           
[7]	validation-rmse:8.72485                           
[8]	validation-rmse:8.47022                           
[9]	validation-rmse:8.24832                           
[10]	validation-rmse:8.04986                          
[11]	validation-rmse:7.87599                          
[12]	validation-rmse:7.72043                          
[13]	validation-rmse:7.58823                          
[14]	validation-rmse:7.46748                          
[15]	validation-rmse:7.36565                          
[16]	validation-rmse:7.27366                          
[17]	validation-rmse:7.19242                          
[18]	valid

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [16:50:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:6.70352                                                     
[3]	validation-rmse:6.67495                                                     
[4]	validation-rmse:6.65601                                                     
[5]	validation-rmse:6.64645                                                     
[6]	validation-rmse:6.64244                                                     
[7]	validation-rmse:6.63597                                                     
[8]	validation-rmse:6.63484                                                     
[9]	validation-rmse:6.63326                                                     
[10]	validation-rmse:6.63100                                                    
[11]	validation-rmse:6.62977                                                    
[12]	validation-rmse:6.62748                                                    
[13]	validation-rmse:6.62678                                                    
[14]	validation-rmse:6.62483

...

[0]	validation-rmse:8.79894                                                     
[1]	validation-rmse:7.40041                                                     
[2]	validation-rmse:6.86558                                                     
[3]	validation-rmse:6.65714                                                     
[4]	validation-rmse:6.56664                                                     
[5]	validation-rmse:6.52321                                                     
[6]	validation-rmse:6.49787                                                     
[7]	validation-rmse:6.48106                                                     
[8]	validation-rmse:6.46938                                                     
[9]	validation-rmse:6.45894                                                     
[10]	validation-rmse:6.45382                                                    
[11]	validation-rmse:6.44900                                                    
[12]	validation-rmse:6.44587

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:04:44] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.02559                                                     
[1]	validation-rmse:6.91736                                                     
[2]	validation-rmse:6.63463                                                     
[3]	validation-rmse:6.53944                                                     
[4]	validation-rmse:6.50648                                                     
[5]	validation-rmse:6.48964                                                     
[6]	validation-rmse:6.47838                                                     
[7]	validation-rmse:6.46931                                                     
[8]	validation-rmse:6.46360                                                     
[9]	validation-rmse:6.45841                                                     
[10]	validation-rmse:6.45090                                                    
[11]	validation-rmse:6.44605                                                    
[12]	validation-rmse:6.44072

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:04:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.17956                                                    
[1]	validation-rmse:10.31614                                                    
[2]	validation-rmse:9.59959                                                     
[3]	validation-rmse:9.00869                                                     
[4]	validation-rmse:8.52485                                                     
[5]	validation-rmse:8.13004                                                     
[6]	validation-rmse:7.80724                                                     
[7]	validation-rmse:7.54853                                                     
[8]	validation-rmse:7.34221                                                     
[9]	validation-rmse:7.17467                                                     
[10]	validation-rmse:7.04037                                                    
[11]	validation-rmse:6.93406                                                    
[12]	validation-rmse:6.84670

In [22]:
# Training model with best hyperparameter
params = {
    "reg_lambda": 0.3065917827492074,
    "seed": 42,
    "max_depth": 44,
    "min_child_weight": 1.5313176894122238,
    "learning_rate": 0.29308959199284523,
    "reg_alpha": 0.04072239461448101,
    "objective": "reg:linear"
}

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)

2024/05/21 17:20:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '74431e38926f4091896d7b487e8e3cfc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:20:40] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.92991
[1]	validation-rmse:8.52297
[2]	validation-rmse:7.68939
[3]	validation-rmse:7.20684
[4]	validation-rmse:6.92636
[5]	validation-rmse:6.76398
[6]	validation-rmse:6.66236
[7]	validation-rmse:6.60186
[8]	validation-rmse:6.56244
[9]	validation-rmse:6.53434
[10]	validation-rmse:6.51456
[11]	validation-rmse:6.49751
[12]	validation-rmse:6.48653
[13]	validation-rmse:6.47770
[14]	validation-rmse:6.47346
[15]	validation-rmse:6.47077
[16]	validation-rmse:6.46818
[17]	validation-rmse:6.46551
[18]	validation-rmse:6.46192
[19]	validation-rmse:6.45806
[20]	validation-rmse:6.45448
[21]	validation-rmse:6.45206
[22]	validation-rmse:6.44945
[23]	validation-rmse:6.44625
[24]	validation-rmse:6.44432
[25]	validation-rmse:6.44208
[26]	validation-rmse:6.44071
[27]	validation-rmse:6.43824
[28]	validation-rmse:6.43566
[29]	validation-rmse:6.43311
[30]	validation-rmse:6.43045
[31]	validation-rmse:6.42843
[32]	validation-rmse:6.42733
[33]	validation-rmse:6.42608
[34]	validation-rmse:6.4

2024/05/21 17:21:06 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/21 17:21:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:21:06] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/05/21 17:21:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or erro

In [28]:
mlflow.xgboost.autolog(disable=True)
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    # Training model with best hyperparameter
    best_params = {
        "reg_lambda": 0.3065917827492074,
        "seed": 42,
        "max_depth": 44,
        "min_child_weight": 1.5313176894122238,
        "learning_rate": 0.29308959199284523,
        "reg_alpha": 0.04072239461448101,
        "objective": "reg:linear"
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=350,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:9.92991


/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:31:20] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:8.52297
[2]	validation-rmse:7.68939
[3]	validation-rmse:7.20684
[4]	validation-rmse:6.92636
[5]	validation-rmse:6.76398
[6]	validation-rmse:6.66236
[7]	validation-rmse:6.60186
[8]	validation-rmse:6.56244
[9]	validation-rmse:6.53434
[10]	validation-rmse:6.51456
[11]	validation-rmse:6.49751
[12]	validation-rmse:6.48653
[13]	validation-rmse:6.47770
[14]	validation-rmse:6.47346
[15]	validation-rmse:6.47077
[16]	validation-rmse:6.46818
[17]	validation-rmse:6.46551
[18]	validation-rmse:6.46192
[19]	validation-rmse:6.45806
[20]	validation-rmse:6.45448
[21]	validation-rmse:6.45206
[22]	validation-rmse:6.44945
[23]	validation-rmse:6.44625
[24]	validation-rmse:6.44432
[25]	validation-rmse:6.44208
[26]	validation-rmse:6.44071
[27]	validation-rmse:6.43824
[28]	validation-rmse:6.43566
[29]	validation-rmse:6.43311
[30]	validation-rmse:6.43045
[31]	validation-rmse:6.42843
[32]	validation-rmse:6.42733
[33]	validation-rmse:6.42608
[34]	validation-rmse:6.42444
[35]	validation-rmse:6.

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:31:31] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutil

In [29]:
logged_model = 'runs:/1a1c78f87a274675b3fe95c8054cb5b5/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:37:14] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [30]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 1a1c78f87a274675b3fe95c8054cb5b5

In [31]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:39:16] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [32]:
y_pred = xgboost_model.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)
rmse

6.304511869194922